# 1- Cuting images in circle

In [25]:
import cv2
import numpy as np

cimg = cv2.imread('olho.jpg')
cimg = cv2.medianBlur(cimg,5)
#cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
output = cv2.cvtColor(cimg, cv2.COLOR_BGR2GRAY)
#height,width = cimg.shape
#mask = np.zeros((height,width), np.uint8)
mask = np.zeros(cimg.shape[:2],np.uint8)
counter = 0

circles = cv2.HoughCircles(output, cv2.HOUGH_GRADIENT, 1.1, 100)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    #cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    #cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    # Draw on mask
    cv2.circle(mask,(i[0],i[1]),i[2],(255,255,255),-1)
    masked_data = cv2.bitwise_and(cimg, cimg, mask=mask)    
    # Apply Threshold
    _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
    # Find Contour
    cnt = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
    #print len(contours)
    x,y,w,h = cv2.boundingRect(cnt[0])
    # Crop masked_data
    crop = masked_data[y:y+h,x:x+w]
    # Write Files
    cv2.imwrite("generated_circles/crop"+str(counter)+".jpg", crop)
    counter +=1
print(counter)
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.imshow("image", cimg)
cv2.resizeWindow('image', 600,600)
cv2.imwrite("generated_circles/circled_img.jpg", cimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

8


# 2- Removing glare

In [26]:
import numpy
import cv2

# prepare mask - square of width 'w' centered at (x,y) which is the center of the bright spots
x = 475
y = 468
w = 100
thresh = 0.9

def remove_glare(im):
   # generate binary image mask - dilated circles around the saturated bright spots at the center
    temp = im[y-w:y+w, x-w:x+w,1]  # single channel
    ret, temp_mask = cv2.threshold(temp, thresh*256, 255, cv2.THRESH_BINARY)
    kernel = numpy.ones((25,25), 'uint8')
    temp_mask = cv2.dilate(temp_mask, kernel)

    # perform the inpainting...
    im[y-w:y+w, x-w:x+w,:] = cv2.inpaint(im[y-w:y+w, x-w:x+w,:], temp_mask, 1, cv2.INPAINT_TELEA)

    # return file
    cv2.namedWindow('image',cv2.WINDOW_NORMAL)
    cv2.imshow("image", im)
    cv2.imwrite("glare_removed/crop0.jpg", im)
    cv2.resizeWindow('image', 600,600)
    cv2.waitKey(0)
    return im
img = cv2.imread('generated_circles/crop0.jpg')

remove_glare(img)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)